<a href="https://colab.research.google.com/github/Spandana2704/ML/blob/main/MLW5(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso

In [17]:
df = pd.read_csv('/content/car_price_prediction.csv')
df = df.dropna(subset=['fiyat'])
numeric_cols = ['yil', 'kilometre', 'motor_hacmi', 'motor_gucu', 'fiyat']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

categorical_cols = ['marka', 'seri', 'model', 'vites_tipi', 'yakit_tipi', 'kasa_tipi', 'renk', 'kimden']
df[categorical_cols] = df[categorical_cols].fillna('Unknown')
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [18]:
y = df_encoded['fiyat']
X_selected = df[['kilometre', 'yil', 'motor_gucu']]
poly = PolynomialFeatures(degree=2)
X_numeric_poly = poly.fit_transform(X_selected)

In [19]:
df_cat_encoded = df_encoded.drop(['kilometre','yil','motor_gucu','fiyat'], axis=1)
df_cat_encoded = df_cat_encoded.apply(pd.to_numeric, errors='coerce')
cat_array = df_cat_encoded.values.astype(float)
X_final = np.concatenate((X_numeric_poly, cat_array), axis=1)

X_final = np.nan_to_num(X_final)

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [28]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled )

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Final R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mse)
print("RMSE:", rmse)
print("Final number of features:", X_final.shape[1])

Final R² Score: 0.07065364180659262
MAE: 312729.0496356146
MSE: 47003705459209.445
RMSE: 6855924.843462729
Final number of features: 3977


In [21]:
ridge = Ridge(alpha=10)
ridge.fit(X_train_scaled, y_train)
ridge_pred = ridge.predict(X_test_scaled)

In [22]:
ridge_mse = mean_squared_error(y_test, ridge_pred)
ridge_rmse = np.sqrt(ridge_mse)

In [23]:
print("\n--- Ridge Regression ---")
print("R² Score:", r2_score(y_test, ridge_pred))
print("MAE:", mean_absolute_error(y_test, ridge_pred))
print("MSE:", ridge_mse)
print("RMSE:", ridge_rmse)
print("Number of non-zero coefficients:", np.sum(ridge.coef_ != 0))


--- Ridge Regression ---
R² Score: 0.06820991216837358
MAE: 307482.27611568273
MSE: 47127302379909.79
RMSE: 6864932.802286545
Number of non-zero coefficients: 3766


In [24]:
lasso = Lasso(alpha=0.1, max_iter=1000)  # Lasso needs more iterations for convergence
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.112e+17, tolerance: 4.920e+13
  model = cd_fast.enet_coordinate_descent(


In [25]:
lasso_mse = mean_squared_error(y_test, lasso_pred)
lasso_rmse = np.sqrt(lasso_mse)

In [26]:
print("\n--- Lasso Regression ---")
print("R² Score:", r2_score(y_test, lasso_pred))
print("MAE:", mean_absolute_error(y_test, lasso_pred))
print("MSE:", lasso_mse)
print("RMSE:", lasso_rmse)
print("Number of non-zero coefficients:", np.sum(lasso.coef_ != 0))


--- Lasso Regression ---
R² Score: 0.08643565241329088
MAE: 305679.2679691992
MSE: 46205496081649.28
RMSE: 6797462.473721299
Number of non-zero coefficients: 3664


In [29]:
print("\n--- Comparison of Models ---")
print(f"Linear Regression MSE: {mse:.2f}")
print(f"Ridge Regression MSE: {ridge_mse:.2f}")
print(f"Lasso Regression MSE: {lasso_mse:.2f}")


--- Comparison of Models ---
Linear Regression MSE: 47003705459209.45
Ridge Regression MSE: 47127302379909.79
Lasso Regression MSE: 46205496081649.28
